In [1]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import json
from time import sleep, perf_counter as pf
import itertools as it

In [2]:
def add_type(df, col, val, type):
    df.loc[(df[col.upper()] == val) & (df['TYPE'] == ''), 'TYPE'] = type

In [36]:
den = pd.read_csv("C:\\Users\\zemdy\\Documents\\csi4142\\Project\\Denver.csv")
# display(den)

In [11]:
van = pd.read_csv("C:\\Users\\zemdy\\Documents\\csi4142\\Project\\vancouver5.csv")
# display(van)

In [38]:
# Drop unneeded columns
den.drop(['INCIDENT_ID', 'OFFENSE_ID', 'OFFENSE_CODE', 'OFFENSE_CODE_EXTENSION', 'FIRST_OCCURRENCE_DATE', 'LAST_OCCURRENCE_DATE', 'GEO_X', 'GEO_Y', 'DISTRICT_ID', 'PRECINCT_ID'], 1, inplace=True)
den['TYPE'] = ''
# display(den)

In [15]:
# van.drop(['X', 'Y'], 1, inplace=True)
# display(van)

In [40]:
den['REPORTED_DATE']=pd.to_datetime(den['REPORTED_DATE'])
den=den.loc[den['REPORTED_DATE'].dt.date <= pd.to_datetime('2017-12-31')]

In [43]:
van.TYPE.unique()

array(['Other Theft', 'Break and Enter Residential/Other', 'Mischief',
       'Break and Enter Commercial', 'Offence Against a Person',
       'Theft from Vehicle',
       'Vehicle Collision or Pedestrian Struck (with Injury)',
       'Vehicle Collision or Pedestrian Struck (with Fatality)',
       'Theft of Vehicle', 'Homicide', 'Theft of Bicycle'], dtype=object)

In [44]:
den.OFFENSE_CATEGORY_ID.unique()

array(['all-other-crimes', 'theft-from-motor-vehicle', 'larceny',
       'auto-theft', 'aggravated-assault', 'other-crimes-against-persons',
       'public-disorder', 'traffic-accident', 'drug-alcohol', 'burglary',
       'sexual-assault', 'robbery', 'arson', 'murder',
       'white-collar-crime'], dtype=object)

In [68]:
den.loc[den['OFFENSE_CATEGORY_ID'] == 'all-other-crimes'].OFFENSE_TYPE_ID.unique()

array(['weapon-unlawful-discharge-of', 'criminal-trespassing',
       'traf-vehicular-assault', 'littering', 'animal-cruelty-to',
       'pawn-broker-viol', 'traf-other', 'public-order-crimes-other',
       'violation-of-restraining-order', 'public-fighting',
       'probation-violation', 'illegal-dumping',
       'police-obstruct-investigation', 'fireworks-possession',
       'gambling-gaming-operation', 'gambling-device',
       'weapon-flourishing', 'police-interference', 'vehicular-eluding',
       'weapon-by-prev-offender-powpo', 'violation-of-court-order',
       'traf-habitual-offender', 'police-resisting-arrest',
       'police-false-information', 'vehicular-eluding-no-chase',
       'intimidation-of-a-witness', 'weapon-carrying-concealed',
       'failure-to-appear', 'kidnap-adult-victim',
       'contraband-into-prison', 'reckless-endangerment',
       'weapon-other-viol', 'agg-aslt-police-weapon', 'escape',
       'weapon-carrying-prohibited', 'bomb-threat',
       'police-d

In [65]:
# Theft
add_type(den, 'offense_category_id', 'theft-from-motor-vehicle', 'Theft from Vehicle')
add_type(den, 'offense_category_id', 'auto-theft', 'Theft of Vehicle')
## Larceny
add_type(den, 'offense_type_id', 'theft-bicycle', 'Theft of Bicycle')
add_type(den, 'offense_category_id', 'larceny', 'Other Theft')
## Robbery
add_type(den, 'offense_type_id', 'robbery-car-jacking', 'Theft of Vehicle')
add_type(den, 'offense_category_id', 'robbery', 'Other Theft')
## Burglary
add_type(den, 'offense_type_id', 'burg-auto-theft-resd-w-force', 'Theft of Vehicle')
add_type(den, 'offense_type_id', 'burg-auto-theft-resd-no-force', 'Theft of Vehicle')
add_type(den, 'offense_type_id', 'burg-auto-theft-busn-w-force', 'Theft of Vehicle')
add_type(den, 'offense_type_id', 'burg-auto-theft-busn-no-force', 'Theft of Vehicle')
add_type(den, 'offense_category_id', 'burglary', 'Other Theft')
## White-collar-crimes
add_type(den, 'offense_type_id', 'theft-embezzle', 'Other Theft')
add_type(den, 'offense_type_id', 'theft-unauth-use-of-ftd', 'Other Theft')
add_type(den, 'offense_type_id', 'theft-fail-return-rent-veh', 'Theft of Vehicle')
add_type(den, 'offense_type_id', 'theft-of-rental-property', 'Other Theft') 

# Homocide
add_type(den, 'offense_type_id', 'traf-vehicular-homicide', 'Vehicle Collision or Pedestrian Struck (with Fatality)')
add_type(den, 'offense_category_id', 'murder', 'Homicide')
add_type(den, 'offense_category_id', 'traffic-accident', 'Vehicle Collision or Pedestrian Struck (with Injury)')

# This one needs to be fixed later.
add_type(den, 'offense_category_id', 'drug-alcohol', 'drug-alcohol_CHANGE_ME')
# Public Disorder 
add_type(den, 'offense_type_id', 'criminal-mischief-other', 'Mischief')
add_type(den, 'offense_type_id', 'criminal-mischief-mtr-veh', 'Mischief')
add_type(den, 'offense_type_id', 'criminal-mischief-graffiti', 'Mischief')
add_type(den, 'offense_type_id', 'riot', 'Mischief')
add_type(den, 'offense_type_id', 'loitering', 'Mischief')
add_type(den, 'offense_type_id', 'prostitution-engaging-in', 'Mischief')
add_type(den, 'offense_type_id', 'prostitution-aiding', 'Mischief')
add_type(den, 'offense_category_id', 'public-disorder', 'Offence Against a Person')

#all-other-crimes
add_type(den, 'offense_type_id', 'criminal-trespassing', 'Mischief')
add_type(den, 'offense_type_id', 'criminal-trespassing', 'Mischief')
add_type(den, 'offense_type_id', 'littering', 'Mischief') 
add_type(den, 'offense_type_id', 'public-fighting', 'Mischief')
add_type(den, 'offense_type_id', 'illegal-dumping', 'Mischief')
add_type(den, 'offense_type_id', 'eavesdropping', 'Mischief')

# display(den.loc[(den['TYPE'] == '') & (den['OFFENSE_CATEGORY_ID'] != 'all-other-crimes')])